In [5]:
import json
import re
from collections import defaultdict

import requests
import pandas as pd
import geopandas as gpd
import networkx as nx
from shapely.geometry import LineString, Point
from arcgis2geojson import arcgis2geojson

# -----------------------------
# SETTINGS
# -----------------------------
AFFECTED_PATH = "Miami-Permits-Affected-Streets_attributes-only.geojson"

# Official Miami-Dade GeoStreets centerlines (FeatureServer/0)
CENTERLINES_LAYER_URL = (
    "https://services.arcgis.com/8Pc9XBTAsYuxx9Ny/arcgis/rest/services/GeoStreets_gdb/FeatureServer/0"
)

# Miami area: meters-based CRS (UTM 17N is practical)
METERS_CRS = 26917

# Your definition: within 50 feet of affected street segments
BUFFER_FEET = 50.0
BUFFER_METERS = BUFFER_FEET * 0.3048

# Snapping and matching tolerances (meters)
ENDPOINT_SNAP_M = 0.25       # for stable graph node keys
NODE_SNAP_TOL_M = 120.0      # snap intersection points to closest main-street node
INTERSECTION_TOL_M = 0.0     # intersection computed geometrically; keep 0

# If you keep getting "missing_cross_intersection", increase NODE_SNAP_TOL_M to 200.


# -----------------------------
# ARCGIS DOWNLOAD (EsriJSON)
# -----------------------------
def arcgis_download_all_esrijson(layer_url: str, where="1=1", out_fields="*", chunk=2000):
    """Download all features from ArcGIS FeatureServer layer as EsriJSON, paginating by resultOffset."""
    feats = []
    offset = 0
    while True:
        params = {
            "f": "json",
            "where": where,
            "outFields": out_fields,
            "returnGeometry": "true",
            "outSR": 4326,
            "resultOffset": offset,
            "resultRecordCount": chunk,
        }
        r = requests.get(f"{layer_url}/query", params=params, timeout=180)
        r.raise_for_status()
        js = r.json()
        batch = js.get("features", [])
        if not batch:
            break
        feats.extend(batch)
        offset += len(batch)
        if len(batch) < chunk:
            break

    # Minimal EsriJSON FeatureSet required by arcgis2geojson
    return {
        "displayFieldName": "",
        "fieldAliases": {},
        "geometryType": "esriGeometryPolyline",
        "spatialReference": {"wkid": 4326},
        "features": feats,
    }


# -----------------------------
# STRING NORMALIZATION
# -----------------------------
_DIR_MAP = {
    "N": "N", "NORTH": "N",
    "S": "S", "SOUTH": "S",
    "E": "E", "EAST": "E",
    "W": "W", "WEST": "W",
    "NE": "NE", "NORTHEAST": "NE",
    "NW": "NW", "NORTHWEST": "NW",
    "SE": "SE", "SOUTHEAST": "SE",
    "SW": "SW", "SOUTHWEST": "SW",
}

_TYPE_MAP = {
    "ST": "ST", "STREET": "ST",
    "AVE": "AVE", "AV": "AVE", "AVENUE": "AVE",
    "BLVD": "BLVD", "BOULEVARD": "BLVD",
    "RD": "RD", "ROAD": "RD",
    "DR": "DR", "DRIVE": "DR",
    "CT": "CT", "COURT": "CT",
    "LN": "LN", "LANE": "LN",
    "TER": "TER", "TERRACE": "TER",
    "PL": "PL", "PLACE": "PL",
    "CIR": "CIR", "CIRCLE": "CIR",
    "HWY": "HWY", "HIGHWAY": "HWY",
    "PKWY": "PKWY", "PARKWAY": "PKWY",
}

def norm_whitespace(s: str) -> str:
    return " ".join(str(s).strip().split())

def norm_token(tok: str) -> str:
    return tok.upper().replace(".", "")

def normalize_street_name(s: str) -> str:
    """
    Normalize tokens, standardize direction + type words, remove punctuation.
    Example: "N.E. 1st Street" -> "NE 1ST ST"
    """
    if s is None or (isinstance(s, float) and pd.isna(s)):
        return ""
    s = norm_whitespace(s)
    s = s.replace(",", " ")
    s = re.sub(r"\s*&\s*", " & ", s)
    toks = [norm_token(t) for t in s.split()]
    out = []
    for t in toks:
        if t in _DIR_MAP:
            out.append(_DIR_MAP[t])
        elif t in _TYPE_MAP:
            out.append(_TYPE_MAP[t])
        else:
            out.append(t)
    return " ".join(out).strip()

def simplify_for_match(s: str) -> str:
    """Secondary normalization: remove extra noise tokens and collapse."""
    s = normalize_street_name(s)
    drop = {"THE", "OF"}
    toks = [t for t in s.split() if t not in drop]
    return " ".join(toks).strip()

def key_variants(raw: str) -> list[str]:
    """
    Generate multiple keys for robust matching.
    Priority order: most specific -> less specific.
    """
    s = simplify_for_match(raw)
    if not s:
        return []
    toks = s.split()
    variants = []

    def add(v):
        v = v.strip()
        if v and v not in variants:
            variants.append(v)

    # 1) full
    add(s)

    # 2) drop trailing type
    if toks and toks[-1] in set(_TYPE_MAP.values()):
        add(" ".join(toks[:-1]))

    # 3) drop leading dir
    if toks and toks[0] in set(_DIR_MAP.values()):
        add(" ".join(toks[1:]))

    # 4) drop both
    tt = toks[:]
    if tt and tt[0] in set(_DIR_MAP.values()):
        tt = tt[1:]
    if tt and tt[-1] in set(_TYPE_MAP.values()):
        tt = tt[:-1]
    if tt:
        add(" ".join(tt))

    return variants


# -----------------------------
# CENTERLINE NAME FIELD DISCOVERY
# -----------------------------
def pick_centerline_name_field(streets: gpd.GeoDataFrame) -> str:
    """
    Try to find the best "full street name" field automatically.
    If none found, fall back to assembling from parts (handled elsewhere).
    """
    candidates = [
        "FULLNAME", "FULL_NAME", "FULL_STREET_NAME",
        "STREETNAME", "STREET_NAME", "RDNAME", "RD_NAME", "NAME"
    ]
    cols_upper = {c.upper(): c for c in streets.columns}
    for want in candidates:
        if want in cols_upper:
            return cols_upper[want]
    return ""


def build_centerline_fullname(streets: gpd.GeoDataFrame) -> pd.Series:
    """
    Build a "best effort" full-name string from common component fields if no full-name field exists.
    """
    # Common-ish component names; your dataset may differ.
    component_candidates = [
        ("PRE_DIR", "PREDIR", "PREFIXDIR", "PRE_DIRECTION"),
        ("ST_NAME", "NAME", "STREETNAME", "RDNAME"),
        ("ST_TYPE", "TYPE", "STREETTYPE", "RD_TYPE"),
        ("SUF_DIR", "SUFDIR", "SUFFIXDIR", "SUF_DIRECTION"),
    ]

    def find_col(options):
        for o in options:
            if o in streets.columns:
                return o
        # also match case-insensitively
        upper_map = {c.upper(): c for c in streets.columns}
        for o in options:
            if o.upper() in upper_map:
                return upper_map[o.upper()]
        return ""

    pre = find_col(component_candidates[0])
    name = find_col(component_candidates[1])
    typ = find_col(component_candidates[2])
    suf = find_col(component_candidates[3])

    if not name:
        raise ValueError(
            "Could not identify a usable street-name field in centerlines.\n"
            f"Columns include: {list(streets.columns)[:60]} ..."
        )

    parts = []
    for col in [pre, name, typ, suf]:
        if col:
            parts.append(streets[col].fillna("").astype(str))
    if not parts:
        raise ValueError("Unexpected: no component fields found even though name existed.")

    full = parts[0]
    for p in parts[1:]:
        full = full.str.cat(p, sep=" ")
    return full.map(normalize_street_name)


# -----------------------------
# GEOMETRY HELPERS
# -----------------------------
def endpoints(line: LineString):
    coords = list(line.coords)
    return Point(coords[0]), Point(coords[-1])

def node_key(pt: Point, snap_m=ENDPOINT_SNAP_M):
    return (round(pt.x / snap_m) * snap_m, round(pt.y / snap_m) * snap_m)

def build_graph(main_gdf_m: gpd.GeoDataFrame) -> nx.Graph:
    """Graph for ONE main street; nodes are snapped endpoints; edges weighted by length."""
    G = nx.Graph()
    for idx, row in main_gdf_m.iterrows():
        geom = row.geometry
        if geom is None or geom.is_empty:
            continue
        a, b = endpoints(geom)
        na, nb = node_key(a), node_key(b)
        G.add_edge(na, nb, fid=idx, weight=float(geom.length))
    return G

def intersection_points(main_gdf_m: gpd.GeoDataFrame, cross_gdf_m: gpd.GeoDataFrame):
    """Return representative points where the main street geometry intersects the cross street geometry."""
    if main_gdf_m.empty or cross_gdf_m.empty:
        return []
    inter = main_gdf_m.unary_union.intersection(cross_gdf_m.unary_union)

    if inter.is_empty:
        return []

    if inter.geom_type == "Point":
        return [inter]
    if inter.geom_type == "MultiPoint":
        return list(inter.geoms)

    # If overlap results in lines/collections, take representative points
    pts = []
    if hasattr(inter, "geoms"):
        for g in inter.geoms:
            if g.is_empty:
                continue
            if g.geom_type == "Point":
                pts.append(g)
            elif g.geom_type in ("LineString", "MultiLineString"):
                pts.append(g.representative_point())
    elif inter.geom_type in ("LineString", "MultiLineString"):
        pts.append(inter.representative_point())

    return pts

def snap_points_to_nodes(G: nx.Graph, pts, tol_m=NODE_SNAP_TOL_M):
    """Snap intersection points to nearest graph nodes within tolerance."""
    nodes = list(G.nodes)
    if not nodes:
        return set()
    out = set()

    for p in pts:
        best_d2 = None
        best_n = None
        for n in nodes:
            d2 = (p.x - n[0])**2 + (p.y - n[1])**2
            if best_d2 is None or d2 < best_d2:
                best_d2 = d2
                best_n = n
        if best_d2 is not None:
            dist = best_d2 ** 0.5
            if dist <= tol_m:
                out.add(best_n)
    return out

def shortest_path_edge_fids(G: nx.Graph, A_nodes, B_nodes):
    """Shortest path between any A and any B; return list of edge fids."""
    best = None
    for a in A_nodes:
        for b in B_nodes:
            if a == b:
                continue
            try:
                path = nx.shortest_path(G, a, b, weight="weight")
                dist = nx.shortest_path_length(G, a, b, weight="weight")
                if best is None or dist < best[0]:
                    best = (dist, path)
            except nx.NetworkXNoPath:
                continue

    if best is None:
        return None

    _, path = best
    fids = []
    for u, v in zip(path[:-1], path[1:]):
        fids.append(G[u][v]["fid"])
    return fids


# -----------------------------
# MAIN
# -----------------------------
def main():
    # 1) Load your attributes-only GeoJSON
    with open(AFFECTED_PATH, "r", encoding="utf-8") as f:
        affected_gj = json.load(f)
    affected_df = pd.DataFrame([ft.get("properties", {}) for ft in affected_gj.get("features", [])])

    # Adjust these if your property names differ
    col_street = "street_full_name"
    col_from = "from_cross_street"
    col_to = "to_cross_street"
    col_id = "id" if "id" in affected_df.columns else None

    if col_street not in affected_df.columns:
        raise ValueError(f"Expected '{col_street}' in your GeoJSON properties. Found: {list(affected_df.columns)}")

    # 2) Download official centerlines and convert to GeoPandas
    print("Downloading Miami-Dade centerlines (this can take a bit)...")
    esri = arcgis_download_all_esrijson(CENTERLINES_LAYER_URL)
    gj = arcgis2geojson(esri)
    streets = gpd.GeoDataFrame.from_features(gj["features"], crs=4326)

    # 3) Decide how to get a "full name" field from centerlines
    name_field = pick_centerline_name_field(streets)

    if name_field:
        print(f"Using centerline name field: {name_field}")
        streets["FULLNAME_NORM"] = streets[name_field].fillna("").astype(str).map(normalize_street_name)
    else:
        print("No obvious full-name field found; assembling from parts...")
        streets["FULLNAME_NORM"] = build_centerline_fullname(streets)

    # 4) Project to meters for routing + buffering
    streets_m = streets.to_crs(METERS_CRS)

    # 5) Build an index mapping variant key -> list of indices (centerlines)
    idx = defaultdict(list)
    for i, val in streets_m["FULLNAME_NORM"].fillna("").astype(str).items():
        for k in key_variants(val):
            idx[k].append(i)

    def best_match_gdf(raw_name: str) -> gpd.GeoDataFrame:
        """Return centerline subset for first matching key variant."""
        for k in key_variants(raw_name):
            hits = idx.get(k, [])
            if hits:
                return streets_m.loc[hits].copy()
        return streets_m.iloc[0:0].copy()

    # 6) Process affected rows: main street between cross streets
    qa_rows = []
    out_segments = []

    for i, rec in affected_df.iterrows():
        rid = rec[col_id] if col_id else i
        raw_main = rec.get(col_street, "")
        raw_from = rec.get(col_from, "")
        raw_to = rec.get(col_to, "")

        main_gdf = best_match_gdf(raw_main)
        from_gdf = best_match_gdf(raw_from)
        to_gdf = best_match_gdf(raw_to)

        if main_gdf.empty:
            qa_rows.append({"id": rid, "status": "no_main_match", "main": raw_main, "from": raw_from, "to": raw_to})
            continue
        if from_gdf.empty or to_gdf.empty:
            qa_rows.append({
                "id": rid,
                "status": "cross_name_no_match",
                "main": raw_main,
                "from": raw_from,
                "to": raw_to,
                "from_match": (not from_gdf.empty),
                "to_match": (not to_gdf.empty),
            })
            continue

        G = build_graph(main_gdf)

        # Intersection points (geometry-based)
        A_pts = intersection_points(main_gdf, from_gdf)
        B_pts = intersection_points(main_gdf, to_gdf)

        if not A_pts or not B_pts:
            qa_rows.append({
                "id": rid,
                "status": "no_geometric_intersection",
                "main": raw_main,
                "from": raw_from,
                "to": raw_to,
                "A_pts": len(A_pts),
                "B_pts": len(B_pts),
            })
            continue

        A_nodes = snap_points_to_nodes(G, A_pts, tol_m=NODE_SNAP_TOL_M)
        B_nodes = snap_points_to_nodes(G, B_pts, tol_m=NODE_SNAP_TOL_M)

        if not A_nodes or not B_nodes:
            qa_rows.append({
                "id": rid,
                "status": "intersection_not_snapped_to_graph",
                "main": raw_main,
                "from": raw_from,
                "to": raw_to,
                "A_nodes": len(A_nodes),
                "B_nodes": len(B_nodes),
            })
            continue

        edge_fids = shortest_path_edge_fids(G, A_nodes, B_nodes)
        if not edge_fids:
            qa_rows.append({"id": rid, "status": "no_path", "main": raw_main, "from": raw_from, "to": raw_to})
            continue

        segs = main_gdf.loc[edge_fids].copy()
        segs["affected_id"] = rid
        segs["street_full_name_src"] = raw_main
        segs["from_cross_src"] = raw_from
        segs["to_cross_src"] = raw_to

        out_segments.append(segs)

        qa_rows.append({
            "id": rid,
            "status": "matched",
            "main": raw_main,
            "from": raw_from,
            "to": raw_to,
            "segments": len(segs),
            "length_m": float(segs.length.sum()),
        })

    qa = pd.DataFrame(qa_rows)
    qa.to_csv("qa_matches.csv", index=False)

    if not out_segments:
        print("No segments matched.")
        print("Open qa_matches.csv and look at the dominant 'status' to see why.")
        print("Two common fixes:")
        print("  - Set NODE_SNAP_TOL_M higher (e.g. 200)")
        print("  - Confirm centerline naming field is correct (print streets.columns)")
        return

    affected_lines_m = gpd.GeoDataFrame(pd.concat(out_segments, ignore_index=True), crs=streets_m.crs)
    affected_lines_m = affected_lines_m.drop_duplicates(subset=["geometry"])

    # 7) 50-ft buffer
    buffer_geom_m = affected_lines_m.buffer(BUFFER_METERS).unary_union
    buffer_gdf_m = gpd.GeoDataFrame([{"geometry": buffer_geom_m}], crs=affected_lines_m.crs)

    # 8) Export for Mapbox (WGS84)
    affected_lines = affected_lines_m.to_crs(4326)
    buffer_gdf = buffer_gdf_m.to_crs(4326)

    affected_lines.to_file("affected_streets_lines.geojson", driver="GeoJSON")
    buffer_gdf.to_file("affected_50ft_buffer.geojson", driver="GeoJSON")

    print("Wrote:")
    print("  - affected_streets_lines.geojson")
    print("  - affected_50ft_buffer.geojson")
    print("  - qa_matches.csv")
    print("")
    print("If match rate is still low, open qa_matches.csv and look at the most common status.")
    print("The script is designed so the QA statuses tell you exactly what to loosen.")

if __name__ == "__main__":
    main()


No obvious full-name field found; assembling from parts...


/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_38819/3619362944.py:269: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  inter = main_gdf_m.unary_union.intersection(cross_gdf_m.unary_union)
/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_38819/3619362944.py:269: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  inter = main_gdf_m.unary_union.intersection(cross_gdf_m.unary_union)
/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_38819/3619362944.py:269: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  inter = main_gdf_m.unary_union.intersection(cross_gdf_m.unary_union)
/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_38819/3619362944.py:269: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  inter = main_gdf_m.unary_union.intersection(c

Wrote:
  - affected_streets_lines.geojson
  - affected_50ft_buffer.geojson
  - qa_matches.csv

If match rate is still low, open qa_matches.csv and look at the most common status.
The script is designed so the QA statuses tell you exactly what to loosen.
